In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from prophet import Prophet
import plotly.io as pio
pio.renderers.default = 'notebook'

import plotly.express as px
import pandas as pd
import numpy as np
import plotly.io as pio

# Ensure Plotly renders in Jupyter Notebook
pio.renderers.default = 'notebook'
import warnings
warnings.filterwarnings("ignore")
import warnings
warnings.filterwarnings("ignore")


import logging

# Suppress logging for cmdstanpy
logging.getLogger("cmdstanpy").setLevel(logging.WARNING)

In [ ]:
data_path = Path("/Users/harsha_gurram/repos/HackUTD-RippleEffect/data")

bold_df = pd.read_csv(data_path/Path("Bold_744H-10_31-11_07.csv"))
cour_df = pd.read_csv(data_path/Path("Courageous_729H-09_25-09_28.csv"))


pipe_dict = {
    "Bold": bold_df,
    "Courageous": cour_df
}

In [ ]:
def stream_data(ticker, pipes):
    results_df = {}
    for key, value in pipes.items():
            
        if ticker>len(value):
            temp_df = value.copy()
        else:
            temp_df = value.iloc[:ticker+1].copy()
        
        #Anamoly Detection
        temp_df['ds'] = value["Time"]
        temp_df['y'] = value["Inj Gas Meter Volume Instantaneous"]
        
        if ticker < 3:
            results_df[key] = {
                "actual": temp_df['y'].iloc[ticker%len(value)], 
                "predicted": temp_df['y'].iloc[ticker%len(value)],
                "anamoly": False
            }
            return results_df

        # Fit a Prophet model
        model = Prophet()
        model.fit(temp_df[['ds', 'y']])

        # Make predictions
        future = model.make_future_dataframe(periods=0)  # Predict only on existing data
        forecast = model.predict(future)

        # Merge predictions back into the original DataFrame
        temp_df['yhat'] = forecast['yhat']
        temp_df['deviation'] = abs(temp_df['y'] - temp_df['yhat'])
        threshold = temp_df['deviation'].quantile(0.995)  # Set a threshold at the 95th percentile
        temp_df['anomaly'] = (temp_df['deviation'] > threshold) & ((temp_df['y'] - temp_df['yhat']) < 0)

        results_df[key] = {
            "actual": temp_df['y'].iloc[ticker%len(value)], 
            "predicted": temp_df['yhat'].iloc[ticker%len(value)],
            "anamoly": temp_df['anomaly'].iloc[ticker%len(value)]
        }

    return results_df    

In [ ]:
stream_data(5, pipe_dict)

In [ ]:
import plotly.graph_objects as go
import random
import time

# Initialize figure
fig = go.Figure()

# Add initial trace
fig.add_trace(go.Scatter(x=[], y=[], mode='lines+markers', name='Live Data'))

# Set layout
fig.update_layout(
    title="Live Scrolling Graph Example",
    xaxis=dict(title="Time", range=[0, 10]),
    yaxis=dict(title="Value", range=[0, 100]),
    template="plotly_dark"
)

# Show initial figure
fig.show()

# Data storage
x_data = []
y_data = []
current_x = 0

# Update graph function
def update_graph():
    global current_x, x_data, y_data

    # Generate new data point

    new_y = stream_data(current_x,pipe_dict)["Bold"]["actual"]  # Replace with real data
    print(new_y)
    current_x += 1
    x_data.append(current_x)
    y_data.append(new_y)

    # Update figure data
    fig.data[0].x = x_data
    fig.data[0].y = y_data

    # Adjust x-axis range for scrolling
    if current_x > 10:
        fig.update_layout(xaxis=dict(range=[current_x - 10, current_x]))

# Live update loop
while True:
    update_graph()
    time.sleep(0.5)  # Adjust refresh rate

In [ ]:
# Create a fake dataset
np.random.seed(42)
data = pd.DataFrame({
    "Category": np.random.choice(["A", "B", "C", "D"], size=50),
    "Value": np.random.randint(10, 100, size=50),
    "Time": pd.date_range("2024-01-01", periods=50)
})

# Create a scatter plot
fig = px.scatter(
    data,
    x="Time",
    y="Value",
    color="Category",
    size="Value",
    title="Sample Plotly Graph with Fake Data",
    labels={"Time": "Time (Days)", "Value": "Random Values"},
    template="plotly_dark"
)

# Display the plot
fig.show()